In [ ]:
import os,requests,pandas as pd
from dotenv import load_dotenv;load_dotenv()

lat,lon=1.3667,103.8
start_date,end_date="2023-07-01","2025-11-03"
f_start_date,f_end_date="2025-10-28","2025-11-11"

a=requests.get("https://archive-api.open-meteo.com/v1/archive",params={"latitude":lat,"longitude":lon,"start_date":start_date,"end_date":end_date,"hourly":"temperature_2m","format":"json","timeformat":"unixtime"}).json()
w=requests.get("https://api.open-meteo.com/v1/forecast",params={"latitude":lat,"longitude":lon,"hourly":"temperature_2m","start_date":f_start_date,"end_date":f_end_date,"format":"json","timeformat":"unixtime"}).json()

df_archive=pd.DataFrame({"ts":pd.to_datetime(a["hourly"]["time"],unit="s"),"temperature_2m":a["hourly"]["temperature_2m"]})
df_weather=pd.DataFrame({"ts":pd.to_datetime(w["hourly"]["time"],unit="s"),"temperature_2m":w["hourly"]["temperature_2m"]})

token=os.getenv("WAQI_TOKEN")
sensor_ids=[1666,1667,1668]
rows=[]
for sid in sensor_ids:
    j=requests.get(f"https://api.waqi.info/feed/@{sid}/",params={"token":token}).json()
    if j.get("status")=="ok":
        d=j["data"];r={"sensor_id":sid,"ts":pd.to_datetime(d["time"]["iso"]),"aqi":d.get("aqi"),"dominentpol":d.get("dominentpol")}
        for k,v in (d.get("iaqi") or {}).items(): r[f"iaqi_{k}"]=v.get("v")
        rows.append(r)
df_air=pd.DataFrame(rows).sort_values(["sensor_id","ts"]).reset_index(drop=True)
